## TF-IDF(Term Frequency-Inverse Document Frequency)
-------------------------------------------------------------------------------------
* 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법 <br>
* 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음 <br>
[참고한 링크](https://wikidocs.net/31698) <br><br>
__(1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수__<br><br>
__(2) df(t) : 특정 단어 t가 등장한 문서의 수__<br>
특정 단어가 문서에서 총 몇 번 등장했는지는 관심가지지 않으며 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가짐.<br>
ex) '바나나'라는 단어가 문서 2에서 1번, 문서 3에서 100번 등장해도 바나나의 df는 2(문서2, 문서3) 
<br><br>
__(3) idf(d, t) : df(t)에 반비례하는 수__<br><br>
$idf(d, t) = log(\frac{n}{1+df(t)})$ <br><br>
값이 기하급수적으로 커지는 것을 막기 위해 log를 취함. 대부분의 프로그래밍 언어에서는 자연로그(ln, 밑이 e인 로그)를 사용함. <br>
여러 문서에서 등장한 단어의 가중치를 낮추는 역할을 함-> 불용어(는, 를, 그, 나는 ...)
<br><br>

* ex) 문서 4개에서 아래 표와 같은 단어가 등장 <br>
||과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|문서1|0|0|0|1|0|1|1|0|0|
|문서2|0|0|0|1|1|0|1|0|0|
|문서3|0|1|1|0|2|0|0|0|0|
|문서4|1|0|0|0|0|0|0|1|1|

__$TF-IDF=TF * IDF$임. 근데 TF는 특정 문서에서의 특정 단어의 등장 횟수이므로 위 표 자체가 TF임. 와 IDF만 구하면 돼!__ <br>
* IDF는 다음 표와 같다 <br>
|단어|IDF(역 문서 빈도)|
|:---:|:---:|
|과일이|ln(4/(1+1)) = 0.693147|
|길고|ln(4/(1+1)) = 0.693147|
|노란|ln(4/(1+1)) = 0.693147|
|먹고|ln(4/(2+1)) = 0.287682|
|바나나|ln(4/(2+1)) = 0.287682|
|사과|ln(4/(1+1)) = 0.693147|
|싶은|ln(4/(2+1)) = 0.287682|
|저는|ln(4/(1+1)) = 0.693147|
|좋아요|ln(4/(1+1)) = 0.693147|

### 1) TF-IDF를 직접 구현

In [1]:
import pandas as pd
from math import log  # IDF 계산을 위해

# docs: 문서
docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

vocab = list(set(w for doc in docs for w in doc.split()))  # docs 리스트에서 각 문장의 공백을 제거하고 중복된 단어를 제거

In [2]:
print(vocab)

['과일이', '노란', '싶은', '길고', '저는', '좋아요', '바나나', '먹고', '사과']


In [4]:
vocab.sort()
print(vocab)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [15]:
# N: 총 문서의 수
N = len(docs)

def tf(t, d):  # 문서 d에서 단어 t의 등장횟수
    return d.count(t)  # list.count(): 리스트에서 해당 원소의 개수를 셈

def idf(t):
    df = 0
    
    # 각 문서에 대해
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t, d)*idf(t)


# TF(DTM) 구하기
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))  # result의 끝에 각 문서별 단어의 등장 횟수를 추가
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [14]:
# IDF 구하기
result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [13]:
# TF-IDF 행렬 출력
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))
        
tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


## 근데 어차피 요즘 프로그래밍 언어에서는 TF-IDF를 이미 다 구현해줌..
### 2) 사이킷런(sklearn) 이용

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력 - 0번: do, 세 번째 문서에서만 등장
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [21]:
# 사이킷런에서 TF-IDF를 자동 계산해주는 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.fit_transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### 3) 케라스(keras) 이용 - 다층 퍼셉트론으로 텍스트 분류하기